<a href="https://colab.research.google.com/github/TillVollmer5/mass_spectroscopy/blob/main/Concentration_calculation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Concentration calculation
This notebook is used to calculate the concentration of phtalates in an environemental sample using an internal standard for quantitation. The Calibration_calculation.ipynb notebook, yields severall of the required files nescessary for the calculations in this notebook. The files *Cal_{n}_analyte_linreg.csv* need to be used which is located in the *calibration_output* folder on the google drive. If this notebook is not applied from my machine in connection with my google drive, either the path to the required files needs to be adabted or changed.
The following code allows for the calculation of the concentration, by using the linear regression form a calibration series and the ratio of the signal of the analyte with the internal standard (area). All the data regarding the calibration are present on my google drive in the folders: *calibration_input* and *calibration_output*, while a new google drive folder will be used to save the final data in as well as a new input folder.

*Google drive folder:*
quantitation_input
quantitation_output

The .csv files containing the peak list from the sample should be saved in the in the input folder. The names should follow a consequent naming sceme:

***Sample_{date_of_analysis}_{tag_bonus_info}_{Sample_Nr}_pl.csv***

Here the Sample number *(Sample_Nr)* is of key importance and needs to be properly documented in the Labjournal, as it is the main way of differentiation.


The following code is used to import all the nescessary libraries and mount the google drive.

In [15]:
import numpy as np
from google.colab import drive
import glob
import pandas as pd
import os
import csv

drive.mount('/content/drive', force_remount = True)

sample_nr_low = 1 #needs to be addapted to 1 nr in the series
sample_nr_high = 2 #needs to be addapted to the last nr of the series

Mounted at /content/drive


The following code is the addapted code from the Calibration_calculation.ipynb and yields the separated in separated .csv files, which will be used to calculate the ratio in a following code segment.

The output file has the following filename and is saved in the quantitation_output folder:

***Sample_{n}_row{i+1}.csv***

In [ ]:
#Loop over all the designated sample files.
for n in range(sample_nr_low, sample_nr_high):
    files = glob.glob(f'/content/drive/My Drive/quantitation_input/*_{n}_pl.csv')

    if not files:
        print(f'Files matching pattern Sample Nr. {n} was not found.')
        break

    else:
        # Loop over each row and save a separate data frame for each
        df = pd.read_csv(files[0], header=4)
        num_rows = len(df)
        
        for i in range(num_rows):
            row_dfs = []
            for file in files:
                df = pd.read_csv(file, header=4, na_values='')

                # Extract the i'th row of the CSV file and append it to the row data frame list
                row = df.iloc[i, :].dropna().astype(str)
                row_dfs.append(row)

            # Concatenate the row data frames along the columns axis and convert to a data frame
            combined_df = pd.concat(row_dfs, axis=1).T

            # Save the combined data frame to a CSV file with a name indicating the row number
            combined_df.to_csv(f'/content/drive/My Drive/quantitation_output/Sample_{n}_row{i+1}.csv', index=False, header=False)
            
            # Convert all values to strings before joining and print the row in the desired format
            formatted_row = '\t'.join(map(str, combined_df.values[0]))
            print(formatted_row)

14.7	14.64	14.76	8899641679.645	22.39	4269043599.688	22.58
17.48	17.43	17.53	11726497950.556	29.51	5937387109.376	31.4
20.79	20.73	20.86	12856490362.673	32.35	6025451308.624	31.87
23.92	23.88	24.0	4783695940.843	12.04	2205044360.583	11.66
29.63	29.57	29.81	1153167025.723	2.9	368569331.242	1.95
32.17	32.13	32.34	198929696.131	0.5	64452863.573	0.34
34.55	34.5	34.7	121997454.463	0.31	38422419.591	0.2


The following code allows to calculate the ratio of the analytes with the internal standard. The output is saved as:

***Sample_{n}_row*_ratio.csv**

In [ ]:
#Loop over all the designated sample files.
for n in range(sample_nr_low, sample_nr_high):
  divisor_file = f'/content/drive/My Drive/quantitation_output/Sample_{n}_row3.csv'
  directory = '/content/drive/My Drive/quantitation_output'
    
  divisor_row = pd.read_csv(divisor_file, sep=',', header=None)
  #print(divisor_row)
  csv_files = glob.glob(os.path.join(directory, f'*_row*.csv'))

    
  # Process each CSV file
  for csv_file in csv_files:
    if csv_file == divisor_file:
      continue  # Skip the divisor file itself

    data_df = pd.read_csv(csv_file, sep=',', header=None)
    #print(data_df)
    data_df = data_df.apply(pd.to_numeric, errors='coerce')  # Convert values to numeric
      
    # Perform division on the data
    divided_data_df = data_df.div(divisor_row, axis='columns')
    #print(divided_data_df)
    #Create a new file name with the "_ratio" tag
    new_file_name = os.path.splitext(csv_file)[0] + '_ratio.csv'

    divided_data_df.to_csv(new_file_name, sep='\t', header=False, index=False)
    print(new_file_name)

/content/drive/My Drive/quantitation_output/Sample_1_row1_ratio.csv
/content/drive/My Drive/quantitation_output/Sample_1_row2_ratio.csv
/content/drive/My Drive/quantitation_output/Sample_1_row4_ratio.csv
/content/drive/My Drive/quantitation_output/Sample_1_row5_ratio.csv
/content/drive/My Drive/quantitation_output/Sample_1_row6_ratio.csv
/content/drive/My Drive/quantitation_output/Sample_1_row7_ratio.csv


The following code takes the ratio files as well as the parameters for the linear regression (analyte specific), and yields a final output file that contains all the information, such as the information of the ratio used for the quantitaiton, the parameters for the linear equation as well as other information such as compound id and other general sample information. 

The final outpur files have the following name format:

***Sample_{n}_quantitation_output.csv***


In [46]:

file_pattern = '/content/drive/My Drive/quantitation_output/Sample_{}_row{}_ratio.csv'
linregs = {}

for i in range(1, 7):
    if i == 3:
        continue
    linreg = glob.glob(f'/content/drive/My Drive/calibration_export/Cal_{i}_analyte_linreg.csv')
    if len(linreg) > 0:
        linregs[i] = pd.read_csv(linreg[0])
    else:
        print(f"No matching file found for Cal_{i}_analyte_linreg.csv")

print(linregs)

output_dir = '/content/drive/My Drive/quantitation_output/'
header_row1 = ['Header1', 'Header2', 'Header3', 'Header4']
header_row2 = ['Descriptor', 'Intercept', 'Slope', 'Value', 'Concentration']

row_descriptors = ['Row A', 'Row B', 'Row C', 'Row D', 'Row E', 'Row F']

for n in range(sample_nr_low, sample_nr_high):
    output_file = output_dir + f'Sample_{n}_concentration_output.csv'

    with open(output_file, 'w', newline='') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow(header_row1)
        writer.writerow(header_row2)

        for i, descriptor in enumerate(row_descriptors):
            files = glob.glob(file_pattern.format(n, i+1))
            for file in files:
                sample_df = pd.read_csv(file, header=None)
                if sample_df.shape[0] >= 1:
                    linreg_df = linregs.get(i + 1)
                    if linreg_df is not None and linreg_df.shape[0] >= 1:
                        intercept_str = linreg_df['Intercept'].values[0]
                        slope_str = linreg_df['Slope'].values[0]

                        if intercept_str and slope_str:
                            intercept = float(re.findall(r'[-\d.]+', intercept_str)[0])
                            slope = float(re.findall(r'[-\d.]+', slope_str)[0])

                            values = sample_df.iloc[0, 0].split('\t')
                            value = float(values[0])
                            concentration = slope * value + intercept

                            row_data = [descriptor, intercept, slope, value, concentration]
                            writer.writerow(row_data)
                        else:
                            print(f"Invalid linreg DataFrame structure for {file}")
                    else:
                        print(f"Invalid linreg DataFrame structure for {file}")
                else:
                    print(f"Empty or invalid sample DataFrame for {file}")


{1:        Intercept           Slope  R2 Value
0  [-0.02459418]  [[0.20015848]]   0.98714, 2:        Intercept           Slope  R2 Value
0  [-0.03101178]  [[0.26080997]]  0.988923, 4:        Intercept           Slope  R2 Value
0  [-0.04441677]  [[0.34518071]]  0.988097, 5:        Intercept           Slope  R2 Value
0  [-0.03514634]  [[0.19182459]]  0.976946, 6:        Intercept           Slope  R2 Value
0  [-0.04753296]  [[0.24848578]]  0.975333}
